Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Shear Kernel Tutorial

This notebook demonstrates how to use the specialized lensing shear kernel (hard-coded to RBF at the moment).

⚠️ _Note that this is still an experimental feature._ ⚠️

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import LogNorm
from MuyGPyS._src.gp.tensors import _crosswise_differences, _pairwise_differences
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2, F2
from MuyGPyS.gp.hyperparameter import Parameter
from MuyGPyS.gp.kernels.experimental import ShearKernel

This is required to import the implementation from Bob Armstrong's original repository.
It must be cloned in the same directory as MuyGPyS for the relative paths to work.

In [ ]:
import importlib.util
import sys
spec_analytic = importlib.util.spec_from_file_location("analytic_kernel", "../../shear_kernel/analytic_kernel.py")
bar = importlib.util.module_from_spec(spec_analytic)
sys.modules["analytic_kernel"] = bar
spec_analytic.loader.exec_module(bar)
from analytic_kernel import shear_kernel

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

Here we build some simple data, which is mean to represent a grid of sky coordinates.

In [ ]:
n = 25  # number of galaxies on a side
xmin = 0
xmax = 1
ymin = 0
ymax = 1

xx = np.linspace(xmin, xmax, n)
yy = np.linspace(ymin, ymax, n)

x, y = np.meshgrid(xx, yy)
features = np.vstack((x.flatten(), y.flatten())).T
data_count = features.shape[0]
diffs = _pairwise_differences(features)
length_scale = 0.5

Use an Isotropic distance functor.

In [ ]:
dist_fn = Isotropy(
    metric=F2,
    length_scale=Parameter(length_scale),
)

Here we construct a shear value kernel (partial differential components of RBF), as well as the original RBF kernel using Bob's implementation.

In [ ]:
def original_shear(X1, X2=None, length_scale=1.0):
    if X2 is None:
        X2 = X1
    n1, _ = X1.shape
    n2, _ = X2.shape
    vals = np.zeros((3 * (n1), 3 * (n2)))
    vals[:] = np.nan
    for i, (ix, iy) in enumerate(X1):
        for j, (jx, jy) in enumerate(X2):
            tmp = shear_kernel(ix, iy, jx, jy, b=length_scale)
            for a in range(3):
                for b in range(3):
                    vals[(a * n1) + i, (b * n2) + j] = tmp[a, b]
            
    return vals

In [ ]:
Kin_analytic = original_shear(features, length_scale=length_scale)

Here we do the same using the MuyGPyS implementation. Note the increased efficiency.

In [ ]:
Kin_muygps = ShearKernel(deformation=dist_fn)(diffs)

`Kin_muygps` is a more generalized tensor, so we need to flatten it to a conforming shape.

In [ ]:
Kin_muygps_flat = Kin_muygps.reshape(data_count * 3, data_count * 3)

In [ ]:
print(f"Kin_analytic.shape = {Kin_analytic.shape}")
print(f"Kin_muygps.shape = {Kin_muygps.shape}")
print(f"Kin_muygps_flat.shape = {Kin_muygps_flat.shape}")

Do the two implementations agree?

In [ ]:
np.allclose(Kin_analytic, Kin_muygps_flat)

In [ ]:
residual = np.abs(Kin_analytic - Kin_muygps_flat)
print(f"residual max: {np.max(residual)}, min: {np.min(residual)}, mean : {np.mean(residual)}")

Plot results of the baseline and internal implementations. 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].set_title("original shear kernel")
axes[0].imshow(Kin_analytic)
axes[1].set_title("MuyGPyS shear kernel")
axes[1].imshow(Kin_muygps_flat)
axes[2].set_title("Residual")
im = axes[2].imshow(residual, norm=LogNorm())
fig.colorbar(im, ax=axes[2])
plt.show()

Now we perform a similar analysis for the cross-covariance.

In [ ]:
split = 200
X1 = features[:split]
X2 = features[split:]
n1, _ = X1.shape
n2, _ = X2.shape
crosswise_diffs = _crosswise_differences(X1, X2)
print(X1.shape, X2.shape, crosswise_diffs.shape)

In [ ]:
Kcross_analytic = original_shear(X1, X2, length_scale=length_scale)

In [ ]:
Kcross_muygps = ShearKernel(deformation=dist_fn)(crosswise_diffs, adjust=False)

In [ ]:
Kcross_muygps_flat = Kcross_muygps.reshape(n1 * 3, n2 * 3)

In [ ]:
print(f"Kcross_analytic.shape = {Kcross_analytic.shape}")
print(f"Kcross_muygps.shape = {Kcross_muygps.shape}")
print(f"Kcross_muygps_flat.shape = {Kcross_muygps_flat.shape}")

In [ ]:
np.allclose(Kcross_analytic, Kcross_muygps_flat)

In [ ]:
cross_residual = np.abs(Kcross_analytic - Kcross_muygps_flat)
print(f"residual max: {np.max(cross_residual)}, min: {np.min(cross_residual)}, mean : {np.mean(cross_residual)}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].set_title("original shear kernel")
axes[0].imshow(Kcross_analytic)
axes[1].set_title("MuyGPyS shear kernel")
axes[1].imshow(Kcross_muygps_flat)
axes[2].set_title("Residual")
im = axes[2].imshow(cross_residual, norm=LogNorm())
fig.colorbar(im, ax=axes[2])
plt.show()

Runtime comparison of the two implementations:

In [ ]:
if False:
    %timeit original_shear(features)
    %timeit ShearKernel(deformation=dist_fn)(diffs)